In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import amp_pd_peptide_310
import numpy as np
import math
import re

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [ ]:
# Load the training datasets into a Pandas DataFrame
# train_proteins = pd.read_csv("train_proteins.csv")
# train_peptides = pd.read_csv("train_peptides.csv")
# train_clinical = pd.read_csv("train_clinical_data.csv")

train_proteins = pd.read_csv("/kaggle/input/amp-parkinsons-disease-progression-prediction/train_proteins.csv")
train_peptides = pd.read_csv("/kaggle/input/amp-parkinsons-disease-progression-prediction/train_peptides.csv")
train_clinical = pd.read_csv("/kaggle/input/amp-parkinsons-disease-progression-prediction/train_clinical_data.csv")

In [ ]:
train_peptides.head()

,visit_id,visit_month,patient_id,UniProt,Peptide,PeptideAbundance
0,55_0,0,55,O00391,NEQEQPLGQWHLS,11254.3
1,55_0,0,55,O00533,GNPEPTFSWTK,102060.0
2,55_0,0,55,O00533,IEIPSSVQQVPTIIK,174185.0
3,55_0,0,55,O00533,KPQSAVYSTGSNGILLC(UniMod_4)EAEGEPQPTIK,27278.9
4,55_0,0,55,O00533,SMEQNGPGLEYR,30838.7


We get all the protein npx values for a given patient visit.

In [ ]:
proteins = train_proteins.pivot(index = ['visit_month','patient_id'], columns = 'UniProt', values = 'NPX')
proteins.head()

UniProt                  O00391    O00533   O00584   O14498    O14773  \
visit_month patient_id                                                  
0           55          11254.3  732430.0  39585.8  41526.9  31238.00   
            1517        11648.9  419015.0  35053.8      NaN  17466.30   
            1923        21361.8  866985.0  32035.1      NaN  13373.10   
            2660            NaN  579829.0  28259.8  29883.2  19134.10   
            3636        10887.0  267402.0  11390.4  15696.3   6646.16   

UniProt                  O14791    O15240   O15394    O43505    O60888  ...  \
visit_month patient_id                                                  ...   
0           55          4202.71  177775.0  62898.2  333376.0  166850.0  ...   
            1517        2784.40  118752.0  40274.9  166322.0   93873.7  ...   
            1923            NaN  269106.0  81580.2  315740.0  297865.0  ...   
            2660        1884.61  134409.0  70664.8  213295.0  229999.0  ...   
            3636        2962.85   61555.3  25927.8   72635.7  110550.0  ...   

UniProt                   Q9HDC9   Q9NQ79    Q9NYU2    Q9UBR2   Q9UBX5  \
visit_month patient_id                                                   
0           55          365475.0  35528.0   97005.6  23122.50  60912.6   
            1517        306280.0  24931.3       NaN  12610.90  40018.9   
            1923         63839.7  58941.2  116485.0  14041.30  66752.8   
            2660        224909.0  32224.2  127529.0  14148.60  26387.0   
            3636        232382.0      NaN   72393.6   7752.62  16678.1   

UniProt                   Q9UHG2   Q9UKV8   Q9UNU6    Q9Y646    Q9Y6R7  
visit_month patient_id                                                  
0           55          408698.0      NaN  29758.8  23833.70  18953.50  
            1517        164826.0  55379.8  31563.0  19771.60  14699.50  
            1923        499634.0  64302.3  25588.8  26346.20  23597.10  
            2660        294153.0  55047.2  17880.5  25630.00  17990.90  
            3636        141754.0      NaN      NaN   7985.07   8587.01  

[5 rows x 227 columns]

For a given patient visit, we have updrs scores and protein data in the merged_df dataframe.

In [ ]:
merged_df = pd.merge(train_clinical,proteins,on = ['visit_month','patient_id'],how = 'inner')

In [ ]:
merged_df.head()

,visit_id,patient_id,visit_month,updrs_1,updrs_2,updrs_3,updrs_4,upd23b_clinical_state_on_medication,O00391,O00533,...,Q9HDC9,Q9NQ79,Q9NYU2,Q9UBR2,Q9UBX5,Q9UHG2,Q9UKV8,Q9UNU6,Q9Y646,Q9Y6R7
0,55_0,55,0,10.0,6.0,15.0,NaN,NaN,11254.3,732430.0,...,365475.0,35528.0,97005.6,23122.5,60912.6,408698.0,NaN,29758.8,23833.7,18953.5
1,55_6,55,6,8.0,10.0,34.0,NaN,NaN,13163.6,630465.0,...,405676.0,30332.6,109174.0,23499.8,51655.8,369870.0,NaN,22935.2,17722.5,16642.7
2,55_12,55,12,10.0,10.0,41.0,0.0,On,15257.6,815083.0,...,303953.0,43026.2,114921.0,21860.1,61598.2,318553.0,65762.6,29193.4,28536.1,19290.9
3,55_36,55,36,17.0,18.0,51.0,0.0,On,13530.8,753832.0,...,303597.0,48188.4,109794.0,23930.6,70223.5,377550.0,74976.1,31732.6,22186.5,21717.1
4,942_6,942,6,8.0,2.0,21.0,NaN,NaN,11218.7,399518.0,...,253373.0,27431.8,93796.7,17450.9,21299.1,306621.0,82335.5,24018.7,18939.5,15251.2


In here, we selected a subset of proteins where these proteins are present in both train and test data.

In [ ]:
f = [ 'visit_id','patient_id','visit_month','updrs_1','updrs_2','updrs_3','updrs_4','upd23b_clinical_state_on_medication','O14791', 'O60888', 'P01009', 'P01608', 'P01860', 'P02753', 'P02763', 'P04156', 'P08294', 'P08603', 'P09486', 'P09871', 'P13521', 'P13611', 'P13987', 'P14174', 'P14618', 'P16035', 'P19021', 'P19823', 'P20774', 'P25311', 'P35542', 'P36222', 'P40925', 'P51884', 'P61626', 'Q06481', 'Q12805', 'Q13740', 'Q14515', 'Q16270', 'Q99969']
for i in list(merged_df.columns):
    if i not in f:
        merged_df = merged_df.drop(i,axis = 1)

In [ ]:
merged_df.shape

(1068, 41)

In [ ]:
merged_df.shape

(1068, 41)

In [ ]:
merged_df.head()

,visit_id,patient_id,visit_month,updrs_1,updrs_2,updrs_3,updrs_4,upd23b_clinical_state_on_medication,O14791,O60888,...,P36222,P40925,P51884,P61626,Q06481,Q12805,Q13740,Q14515,Q16270,Q99969
0,55_0,55,0,10.0,6.0,15.0,NaN,NaN,4202.71,166850.0,...,502305.0,31201.3,380950.0,23749.4,39504.7,1611130.0,34672.4,512157.0,1451950.0,22021.6
1,55_6,55,6,8.0,10.0,34.0,NaN,NaN,4416.42,170345.0,...,407266.0,20023.9,375357.0,24705.3,35475.4,1483200.0,32822.1,393979.0,1443340.0,23109.0
2,55_12,55,12,10.0,10.0,41.0,0.0,On,4343.60,151194.0,...,651428.0,34174.4,396981.0,26521.3,40011.4,1547140.0,37992.3,616993.0,1381090.0,22730.6
3,55_36,55,36,17.0,18.0,51.0,0.0,On,5367.06,122902.0,...,611576.0,25889.8,275869.0,30129.0,41285.5,1725030.0,42288.2,524264.0,1649010.0,23715.0
4,942_6,942,6,8.0,2.0,21.0,NaN,NaN,2564.37,85272.7,...,164113.0,23134.0,295912.0,12999.3,31363.6,637932.0,19047.6,359069.0,780395.0,13060.3


We added an an interesting feature that is the measure of oxidation grade for a protein. This could be computed as an oxidized-to-total peptide abundance ratio.

In [ ]:
# Filter the peptides DataFrame to select only oxidized peptides (UniMod_35 modifications)
oxidized_peptides = train_peptides[train_peptides['Peptide'].str.contains(r'\(UniMod_35\)')]

# Group the peptides by visit month, patient ID, and UniProt
grouped_peptides = oxidized_peptides.groupby(['visit_month', 'patient_id', 'UniProt']).agg({
    'PeptideAbundance': 'sum',
    'Peptide': 'count'
}).reset_index()

# Compute the oxidation grade as the ratio of oxidized peptide count to total peptide count
grouped_peptides['OxidationGrade'] = grouped_peptides['Peptide'] / grouped_peptides.groupby(['visit_month', 'patient_id'])['Peptide'].transform('sum')

# Group the peptides by visit month and patient ID and calculate the average oxidation grade per protein
average_oxidation_grade = grouped_peptides.groupby(['visit_month', 'patient_id'])['OxidationGrade'].mean().reset_index()

merged_df = pd.merge(merged_df,average_oxidation_grade, on = ['patient_id','visit_month'], how = 'left')
merged_df.head()

,visit_id,patient_id,visit_month,updrs_1,updrs_2,updrs_3,updrs_4,upd23b_clinical_state_on_medication,O14791,O60888,...,P40925,P51884,P61626,Q06481,Q12805,Q13740,Q14515,Q16270,Q99969,OxidationGrade
0,55_0,55,0,10.0,6.0,15.0,NaN,NaN,4202.71,166850.0,...,31201.3,380950.0,23749.4,39504.7,1611130.0,34672.4,512157.0,1451950.0,22021.6,0.052632
1,55_6,55,6,8.0,10.0,34.0,NaN,NaN,4416.42,170345.0,...,20023.9,375357.0,24705.3,35475.4,1483200.0,32822.1,393979.0,1443340.0,23109.0,0.052632
2,55_12,55,12,10.0,10.0,41.0,0.0,On,4343.60,151194.0,...,34174.4,396981.0,26521.3,40011.4,1547140.0,37992.3,616993.0,1381090.0,22730.6,0.052632
3,55_36,55,36,17.0,18.0,51.0,0.0,On,5367.06,122902.0,...,25889.8,275869.0,30129.0,41285.5,1725030.0,42288.2,524264.0,1649010.0,23715.0,0.052632
4,942_6,942,6,8.0,2.0,21.0,NaN,NaN,2564.37,85272.7,...,23134.0,295912.0,12999.3,31363.6,637932.0,19047.6,359069.0,780395.0,13060.3,0.055556


In [ ]:
columns = list(merged_df.columns)[8:]

In [ ]:
print(columns)

['O14791', 'O60888', 'P01009', 'P01608', 'P01860', 'P02753', 'P02763', 'P04156', 'P08294', 'P08603', 'P09486', 'P09871', 'P13521', 'P13611', 'P13987', 'P14174', 'P14618', 'P16035', 'P19021', 'P19823', 'P20774', 'P25311', 'P35542', 'P36222', 'P40925', 'P51884', 'P61626', 'Q06481', 'Q12805', 'Q13740', 'Q14515', 'Q16270', 'Q99969', 'OxidationGrade']


We replaced outliers with median when the distribution is skewed, otherwise with mean.

In [ ]:
from scipy.stats import shapiro

# Function to replace outliers with median or mean based on distribution
def replace_outliers(data):
    data_copy = data.copy()  # Create a copy of the data
    median = data_copy.median()
    mean = data_copy.mean()
    is_skewed = shapiro(data_copy)[1] < 0.1  # Shapiro-Wilk test for skewness
    k = 1.5
    if is_skewed:
        data_copy[data_copy < median - k * data_copy.std()] = median
        data_copy[data_copy > median + k * data_copy.std()] = median
    else:
        data_copy[data_copy < mean - k * data_copy.std()] = mean
        data_copy[data_copy > mean + k * data_copy.std()] = mean
    
    return data_copy

# Replace outliers in each column based on distribution
for column in columns:
    merged_df[column] = replace_outliers(merged_df[column])

In [ ]:
merged_df.head()

,visit_id,patient_id,visit_month,updrs_1,updrs_2,updrs_3,updrs_4,upd23b_clinical_state_on_medication,O14791,O60888,...,P40925,P51884,P61626,Q06481,Q12805,Q13740,Q14515,Q16270,Q99969,OxidationGrade
0,55_0,55,0,10.0,6.0,15.0,NaN,NaN,4202.710000,166850.0,...,31201.3,259610.147235,14575.771222,22715.368432,953865.5,34672.400000,512157.000000,892771.8388,12805.383159,0.052632
1,55_6,55,6,8.0,10.0,34.0,NaN,NaN,4416.420000,170345.0,...,20023.9,259610.147235,14575.771222,35475.400000,953865.5,32822.100000,393979.000000,892771.8388,12805.383159,0.052632
2,55_12,55,12,10.0,10.0,41.0,0.0,On,4343.600000,151194.0,...,34174.4,259610.147235,14575.771222,22715.368432,953865.5,21194.646456,378951.339062,892771.8388,12805.383159,0.052632
3,55_36,55,36,17.0,18.0,51.0,0.0,On,3013.701634,122902.0,...,25889.8,275869.000000,14575.771222,22715.368432,953865.5,21194.646456,524264.000000,892771.8388,12805.383159,0.052632
4,942_6,942,6,8.0,2.0,21.0,NaN,NaN,2564.370000,85272.7,...,23134.0,295912.000000,12999.300000,31363.600000,637932.0,19047.600000,359069.000000,780395.0000,13060.300000,0.055556


We replaced NaN with 0 indication the absense of protein for the test for training and also 0 was common in updrs_4 and updrs_3 so we replaced NaN in those columns with 0 as well.

In [ ]:

def replace_nan(data):
    filled_data = data.fillna(0)
    return filled_data

# Replace NaN values in each column with 0
for column in list(merged_df.columns):
    filled_data = replace_nan(merged_df[column])
    merged_df[column] = filled_data



In [ ]:
merged_df.head()

,visit_id,patient_id,visit_month,updrs_1,updrs_2,updrs_3,updrs_4,upd23b_clinical_state_on_medication,O14791,O60888,...,P40925,P51884,P61626,Q06481,Q12805,Q13740,Q14515,Q16270,Q99969,OxidationGrade
0,55_0,55,0,10.0,6.0,15.0,0.0,0,4202.710000,166850.0,...,31201.3,259610.147235,14575.771222,22715.368432,953865.5,34672.400000,512157.000000,892771.8388,12805.383159,0.052632
1,55_6,55,6,8.0,10.0,34.0,0.0,0,4416.420000,170345.0,...,20023.9,259610.147235,14575.771222,35475.400000,953865.5,32822.100000,393979.000000,892771.8388,12805.383159,0.052632
2,55_12,55,12,10.0,10.0,41.0,0.0,On,4343.600000,151194.0,...,34174.4,259610.147235,14575.771222,22715.368432,953865.5,21194.646456,378951.339062,892771.8388,12805.383159,0.052632
3,55_36,55,36,17.0,18.0,51.0,0.0,On,3013.701634,122902.0,...,25889.8,275869.000000,14575.771222,22715.368432,953865.5,21194.646456,524264.000000,892771.8388,12805.383159,0.052632
4,942_6,942,6,8.0,2.0,21.0,0.0,0,2564.370000,85272.7,...,23134.0,295912.000000,12999.300000,31363.600000,637932.0,19047.600000,359069.000000,780395.0000,13060.300000,0.055556


In [ ]:
features = [list(merged_df.columns)[2]] + list(merged_df.columns)[8:]
print(features)

['visit_month', 'O14791', 'O60888', 'P01009', 'P01608', 'P01860', 'P02753', 'P02763', 'P04156', 'P08294', 'P08603', 'P09486', 'P09871', 'P13521', 'P13611', 'P13987', 'P14174', 'P14618', 'P16035', 'P19021', 'P19823', 'P20774', 'P25311', 'P35542', 'P36222', 'P40925', 'P51884', 'P61626', 'Q06481', 'Q12805', 'Q13740', 'Q14515', 'Q16270', 'Q99969', 'OxidationGrade']


In [ ]:
col_y = list(merged_df.columns)[3:7]
print(col_y)

['updrs_1', 'updrs_2', 'updrs_3', 'updrs_4']


We calculated MI score for selected features agains target variables and selected features that have MI score higher than 0.2

In [ ]:
from sklearn.feature_selection import mutual_info_regression

updrs = []
# Assuming merged_df is your dataframe and features and col_y are your lists
for y_col in col_y:
    mi_scores = mutual_info_regression(merged_df[features], merged_df[y_col],random_state=42)
    
    print(f"MI scores for target variable '{y_col}':")
    temp = []
    for i, feature in enumerate(features):
        print(f"{feature}: {mi_scores[i]}")
        if mi_scores[i] >= 0.02:
          temp.append(feature)
    updrs.append(temp)
    print("\n")

MI scores for target variable 'updrs_1':
visit_month: 0.0
O14791: 0.00620039431082553
O60888: 0.039414048718318107
P01009: 0.043262595500807244
P01608: 0.0425588858256738
P01860: 0.044074201875588415
P02753: 0.02661851183326025
P02763: 0.0556919360901329
P04156: 0.053624033378175895
P08294: 0.025363690842323372
P08603: 0.03152847948254678
P09486: 0.05415528217226662
P09871: 0.02988025795901539
P13521: 0.05506505249284377
P13611: 0.06534402956503627
P13987: 0.041201444378187624
P14174: 0.044873896559376014
P14618: 0.06285013337105205
P16035: 0.038163929957427456
P19021: 0.03492257711376201
P19823: 0.04104851400635745
P20774: 0.0445053972292202
P25311: 0.042981774445105536
P35542: 0.05850690755079846
P36222: 0.014185956679722445
P40925: 0.058766631494704846
P51884: 0.042529559553908136
P61626: 0.01411713691842209
Q06481: 0.05542098828497011
Q12805: 0.06200602900719865
Q13740: 0.029578823280596023
Q14515: 0.028410479981062053
Q16270: 0.025222471880872455
Q99969: 0.01689812050349193
Oxidat

Here are the final selected features for training.

In [ ]:
col_x_1 = [features[0]] + updrs[0]
col_x_2 = updrs[1]
col_x_3 = updrs[2]
col_x_4 = updrs[3]

print(col_x_1)
print(col_x_2)
print(col_x_3)
print(col_x_4)

['visit_month', 'O60888', 'P01009', 'P01608', 'P01860', 'P02753', 'P02763', 'P04156', 'P08294', 'P08603', 'P09486', 'P09871', 'P13521', 'P13611', 'P13987', 'P14174', 'P14618', 'P16035', 'P19021', 'P19823', 'P20774', 'P25311', 'P35542', 'P40925', 'P51884', 'Q06481', 'Q12805', 'Q13740', 'Q14515', 'Q16270']
['visit_month', 'O60888', 'P01608', 'P01860', 'P04156', 'P08294', 'P16035', 'P61626', 'Q06481', 'Q12805', 'Q14515', 'OxidationGrade']
['visit_month', 'P04156', 'P08603', 'P13987', 'P14618', 'P16035', 'P19021', 'P36222', 'P40925', 'P51884', 'Q06481', 'Q99969', 'OxidationGrade']
['visit_month', 'P01009', 'P08294', 'P08603', 'P13611', 'P19021']


In [ ]:
print(features)

['visit_month', 'O14791', 'O60888', 'P01009', 'P01608', 'P01860', 'P02753', 'P02763', 'P04156', 'P08294', 'P08603', 'P09486', 'P09871', 'P13521', 'P13611', 'P13987', 'P14174', 'P14618', 'P16035', 'P19021', 'P19823', 'P20774', 'P25311', 'P35542', 'P36222', 'P40925', 'P51884', 'P61626', 'Q06481', 'Q12805', 'Q13740', 'Q14515', 'Q16270', 'Q99969', 'OxidationGrade']


In [ ]:
merged_df.shape

(1068, 42)

In [ ]:
merged_df = merged_df.drop_duplicates()
merged_df.shape

(1068, 42)

Scale all the features except visit month

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

# Apply Min-Max scaling to the selected columns
merged_df[features[1:]] = scaler.fit_transform(merged_df[features[1:]])

merged_df.head()

,visit_id,patient_id,visit_month,updrs_1,updrs_2,updrs_3,updrs_4,upd23b_clinical_state_on_medication,O14791,O60888,...,P40925,P51884,P61626,Q06481,Q12805,Q13740,Q14515,Q16270,Q99969,OxidationGrade
0,55_0,55,0,10.0,6.0,15.0,0.0,0,1.429587,0.773461,...,0.618772,0.133565,0.317439,0.176828,0.105906,1.768589,1.793681,0.189542,0.391035,-0.876169
1,55_6,55,6,8.0,10.0,34.0,0.0,0,1.600091,0.851867,...,-0.482989,0.133565,0.317439,1.599570,0.105906,1.591484,0.272902,0.189542,0.391035,-0.876169
2,55_12,55,12,10.0,10.0,41.0,0.0,On,1.541993,0.422240,...,0.911832,0.133565,0.317439,0.176828,0.105906,0.478541,0.079518,0.189542,0.391035,-0.876169
3,55_36,55,36,17.0,18.0,51.0,0.0,On,0.480961,-0.212451,...,0.095215,0.474120,0.317439,0.176828,0.105906,0.478541,1.949481,0.189542,0.391035,-0.876169
4,942_6,942,6,8.0,2.0,21.0,0.0,0,0.122471,-1.056612,...,-0.176425,0.893937,-0.007947,1.141105,-1.452082,0.273033,-0.176339,-0.335633,0.446657,-0.296078


Drop the column upd23b_clinical_state_on_medication because high abundance of NaN values.

In [ ]:
merged_df = merged_df.drop(columns = 'upd23b_clinical_state_on_medication')
merged_df.head()

,visit_id,patient_id,visit_month,updrs_1,updrs_2,updrs_3,updrs_4,O14791,O60888,P01009,...,P40925,P51884,P61626,Q06481,Q12805,Q13740,Q14515,Q16270,Q99969,OxidationGrade
0,55_0,55,0,10.0,6.0,15.0,0.0,1.429587,0.773461,1.323211,...,0.618772,0.133565,0.317439,0.176828,0.105906,1.768589,1.793681,0.189542,0.391035,-0.876169
1,55_6,55,6,8.0,10.0,34.0,0.0,1.600091,0.851867,0.737182,...,-0.482989,0.133565,0.317439,1.599570,0.105906,1.591484,0.272902,0.189542,0.391035,-0.876169
2,55_12,55,12,10.0,10.0,41.0,0.0,1.541993,0.422240,0.680313,...,0.911832,0.133565,0.317439,0.176828,0.105906,0.478541,0.079518,0.189542,0.391035,-0.876169
3,55_36,55,36,17.0,18.0,51.0,0.0,0.480961,-0.212451,1.020286,...,0.095215,0.474120,0.317439,0.176828,0.105906,0.478541,1.949481,0.189542,0.391035,-0.876169
4,942_6,942,6,8.0,2.0,21.0,0.0,0.122471,-1.056612,0.919185,...,-0.176425,0.893937,-0.007947,1.141105,-1.452082,0.273033,-0.176339,-0.335633,0.446657,-0.296078


Train using Random Fores Regressor and testing with various metrics.

In [ ]:
x_1 = merged_df[col_x_1]
y_1 = merged_df['updrs_1']

x_2 = merged_df[col_x_2]
y_2 = merged_df['updrs_2']

x_3 = merged_df[col_x_3]
y_3 = merged_df['updrs_3']

x_4 = merged_df[col_x_4]
y_4 = merged_df['updrs_4']

In [ ]:
from sklearn.model_selection import train_test_split

test_size = 0.1

x_1_train, x_1_val, y_1_train, y_1_val = train_test_split(x_1,y_1, test_size=test_size, random_state=0)
x_2_train, x_2_val, y_2_train, y_2_val = train_test_split(x_2,y_2, test_size=test_size, random_state=0)
x_3_train, x_3_val, y_3_train, y_3_val = train_test_split(x_3,y_3, test_size=test_size, random_state=0)
x_4_train, x_4_val, y_4_train, y_4_val = train_test_split(x_4,y_3, test_size=test_size, random_state=0)

In [ ]:
def smape(A, F):
    return 100/len(A) * np.sum(2 * np.abs(F - A) / (np.abs(A) + np.abs(F)))

from sklearn.metrics import mean_squared_error, mean_absolute_error

In [ ]:
from sklearn.ensemble import RandomForestRegressor

def grid_search_hyperparameters(x_train,y_train, x_val, y_val, n_est_range, depth_range):
    least_err_n_estimate = 1000
    best_n_estimate = 0
    best_depth = 0
    for n_estimate in n_est_range:
        least_err_depth = 1000
        for depth in depth_range:
            model = RandomForestRegressor(n_estimators = n_estimate, max_depth=depth, random_state=0)
            model.fit(x_train,y_train)
            y_val_pred = model.predict(x_val)
            err = smape(y_val, y_val_pred)
            if err < least_err_depth:
                best_depth = depth
                least_err_depth = err
        if least_err_depth < least_err_n_estimate:
            best_n_estimate = n_estimate
            least_err_n_estimate = least_err_depth
    return best_n_estimate, best_depth, least_err_n_estimate

In [ ]:
n_est_1, depth_1, least_err_1 = grid_search_hyperparameters(x_1_train, y_1_train, x_1_val, y_1_val, range(175,225,5), range(30,40))
n_est_1, depth_1, least_err_1

(195, 33, 64.56517081253074)

In [ ]:
n_est_2, depth_2, least_err_2 = grid_search_hyperparameters(x_2_train, y_2_train, x_2_val, y_2_val, range(50,75,5), range(3,8))
n_est_2, depth_2, least_err_2

(55, 4, 91.20384913653358)

In [ ]:
n_est_3, depth_3, least_err_3 = grid_search_hyperparameters(x_3_train, y_3_train, x_3_val, y_3_val, range(75,125,5), range(15,30))
n_est_3, depth_3, least_err_3

(105, 17, 85.44874033911276)

In [ ]:
n_est_4, depth_4, least_err_4 = grid_search_hyperparameters(x_4_train, y_4_train, x_4_val, y_4_val, range(675,725,5), range(3,8))
n_est_4, depth_4, least_err_4

(695, 3, 86.39272108025209)

In [ ]:
def mean_squared_error(y_true, y_pred):
    """
    Calculates the Mean Squared Error (MSE) between the true and predicted values.

    Parameters:
    - y_true: numpy array or list, true values
    - y_pred: numpy array or list, predicted values

    Returns:
    - mse: float, mean squared error
    """
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    mse = np.mean((y_true - y_pred) ** 2)
    return mse

In [ ]:
def r_squared(y_true, y_pred):
    """
    Calculates the R-squared (coefficient of determination) between the true and predicted values.

    Parameters:
    - y_true: numpy array or list, true values
    - y_pred: numpy array or list, predicted values

    Returns:
    - r2: float, R-squared value
    """
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    ss_total = np.sum((y_true - np.mean(y_true))**2)
    ss_residual = np.sum((y_true - y_pred)**2)
    r2 = 1 - (ss_residual / ss_total)
    return r2

In [ ]:
model1 = RandomForestRegressor(n_estimators=n_est_1, max_depth=depth_1, random_state=42)
model2 = RandomForestRegressor(n_estimators=n_est_2, max_depth=depth_2, random_state=42)
model3 = RandomForestRegressor(n_estimators=n_est_3, max_depth=depth_3, random_state=42)
model4 = RandomForestRegressor(n_estimators=n_est_4, max_depth=depth_4, random_state=42)

model1.fit(x_1_train,y_1_train)
model2.fit(x_2_train,y_2_train)
model3.fit(x_3_train,y_3_train)
model4.fit(x_4_train,y_4_train)

RandomForestRegressor(max_depth=3, n_estimators=695, random_state=42)

In [ ]:
y_1_train_pred = model1.predict(x_1_train)
y_1_val_pred = model1.predict(x_1_val)
y_2_train_pred = model2.predict(x_2_train)
y_2_val_pred = model2.predict(x_2_val)
y_3_train_pred = model3.predict(x_3_train)
y_3_val_pred = model3.predict(x_3_val)
y_4_train_pred = model4.predict(x_4_train)
y_4_val_pred = model4.predict(x_4_val)

In [ ]:
def evaluate_model(y_train, y_train_pred, y_val, y_val_pred):
    temp_df = pd.DataFrame(columns = ['train_error','val_error'], index=['smape','rmse','mae'])
    
    temp_df.loc['smape','train_error'] = smape(y_train, y_train_pred)
    temp_df.loc['smape','val_error'] = smape(y_val, y_val_pred)
    
    temp_df.loc['rmse','train_error'] = (mean_squared_error(y_train, y_train_pred))**(0.5)
    temp_df.loc['rmse','val_error'] = (mean_squared_error(y_val, y_val_pred))**(0.5)
    
    temp_df.loc['mae','train_error'] = mean_absolute_error(y_train, y_train_pred)
    temp_df.loc['mae','val_error'] = mean_absolute_error(y_val, y_val_pred)

    return temp_df

In [ ]:
# for updrs_1
evaluate_model(y_1_train, y_1_train_pred, y_1_val, y_1_val_pred)

,train_error,val_error
smape,38.847638,65.137106
rmse,1.895679,4.23724
mae,1.465135,3.44994


In [ ]:
# for updrs_2
evaluate_model(y_2_train, y_2_train_pred, y_2_val, y_2_val_pred)

,train_error,val_error
smape,93.304062,91.936876
rmse,5.315312,5.03935
mae,4.301161,4.179296


In [ ]:
# for updrs_3
evaluate_model(y_3_train, y_3_train_pred, y_3_val, y_3_val_pred)

,train_error,val_error
smape,63.172353,86.565356
rmse,5.487965,13.467032
mae,4.527872,11.251656


In [ ]:
# for updrs_4
evaluate_model(y_4_train, y_4_train_pred, y_4_val, y_4_val_pred)

,train_error,val_error
smape,87.399096,86.400965
rmse,14.167444,14.144571
mae,11.808006,11.817998


In [ ]:
# for overall data and predictions
comb_y_train = pd.Series(y_1_train.append(y_2_train.append(y_3_train.append(y_4_train))))
comb_y_val = pd.Series(y_1_val.append(y_2_val.append(y_3_val.append(y_4_val))))
comb_y_train_pred = np.concatenate((y_1_train_pred, y_2_train_pred, y_3_train_pred, y_4_train_pred))
comb_y_val_pred = np.concatenate((y_1_val_pred, y_2_val_pred, y_3_val_pred, y_4_val_pred))

evaluate_model(comb_y_train, comb_y_train_pred, comb_y_val, comb_y_val_pred)

/tmp/ipykernel_21/372339784.py:2: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  comb_y_train = pd.Series(y_1_train.append(y_2_train.append(y_3_train.append(y_4_train))))
/tmp/ipykernel_21/372339784.py:2: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  comb_y_train = pd.Series(y_1_train.append(y_2_train.append(y_3_train.append(y_4_train))))
/tmp/ipykernel_21/372339784.py:2: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  comb_y_train = pd.Series(y_1_train.append(y_2_train.append(y_3_train.append(y_4_train))))
/tmp/ipykernel_21/372339784.py:3: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  comb_y_val = pd.Series(y_1_val.append(y_2_val.appe

,train_error,val_error
smape,70.680787,82.510076
rmse,8.103708,10.305085
mae,5.525543,7.674722


In [ ]:
model = {"updrs_1":model1,"updrs_2":model2,"updrs_3":model3,"updrs_4":model4}
features_selected = {"updrs_1":col_x_1,"updrs_2":col_x_2,"updrs_3":col_x_3,"updrs_4":col_x_4}
target = ["updrs_1", "updrs_2", "updrs_3", "updrs_4"]

In [ ]:
def get_predictions(my_train, pro,pep, model):
    proteins = pro.pivot(index=['visit_month', 'patient_id'], columns='UniProt', values='NPX')
    my_train = pd.merge(my_train, proteins, on=['visit_month', 'patient_id'], how='left')
    f = [ 'visit_id','patient_id','visit_month','updrs_1','updrs_2','updrs_3','updrs_4','upd23b_clinical_state_on_medication','O14791', 'O60888', 'P01009', 'P01608', 'P01860', 'P02753', 'P02763', 'P04156', 'P08294', 'P08603', 'P09486', 'P09871', 'P13521', 'P13611', 'P13987', 'P14174', 'P14618', 'P16035', 'P19021', 'P19823', 'P20774', 'P25311', 'P35542', 'P36222', 'P40925', 'P51884', 'P61626', 'Q06481', 'Q12805', 'Q13740', 'Q14515', 'Q16270', 'Q99969']
    for i in list(my_train.columns):
        if i not in f:
            my_train = my_train.drop(i,axis = 1)
            
    # Filter the peptides DataFrame to select only oxidized peptides (UniMod_35 modifications)
    oxidized_peptides = pep[pep['Peptide'].str.contains(r'\(UniMod_35\)')]

    # Group the peptides by visit month, patient ID, and UniProt
    grouped_peptides = oxidized_peptides.groupby(['visit_month', 'patient_id', 'UniProt']).agg({
        'PeptideAbundance': 'sum',
        'Peptide': 'count'
    }).reset_index()

    # Compute the oxidation grade as the ratio of oxidized peptide count to total peptide count
    grouped_peptides['OxidationGrade'] = grouped_peptides['Peptide'] / grouped_peptides.groupby(['visit_month', 'patient_id'])['Peptide'].transform('sum')

    # Group the peptides by visit month and patient ID and calculate the average oxidation grade per protein
    average_oxidation_grade = grouped_peptides.groupby(['visit_month', 'patient_id'])['OxidationGrade'].mean().reset_index()

    my_train = pd.merge(my_train,average_oxidation_grade, on = ['patient_id','visit_month'], how = 'left')
    
    # Check which columns from features are not in my_train
    new_columns = [col for col in features if col not in my_train.columns]

    # Add the new columns to my_train with default value 0
    my_train[new_columns] = 0

    for column in list(my_train.columns):
      filled_data = replace_nan(my_train[column])
      my_train[column] = filled_data

    my_train[features[1:]] = scaler.transform(my_train[features[1:]])

    for u in target:
        # Here is where we will save the final results
        my_train['result_' + str(u)] = 0

        # Predict    
        X = my_train[features_selected[u]]
        if u == "updrs_4":
          my_train['result_' + str(u)] = 0
        else:
          my_train['result_' + str(u)] = np.floor(model[u].predict(X.values))

    # Format for final submission
    result = pd.DataFrame()

    for m in [0, 6, 12, 24]:
        for u in [1, 2, 3, 4]:
            temp = my_train[["visit_id", "result_updrs_" + str(u)]]
            temp["prediction_id"] = temp["visit_id"] + "_updrs_" + str(u) + "_plus_" + str(m) + "_months"
            temp["rating"] = temp["result_updrs_" + str(u)]
            temp = temp[['prediction_id', 'rating']]

            result = result.append(temp)
    result = result.drop_duplicates(subset=['prediction_id', 'rating'])

    return result

# Run once to check results
pred = get_predictions(train_clinical, train_proteins,train_peptides, model)


/opt/conda/lib/python3.10/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
/tmp/ipykernel_21/559174679.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp["prediction_id"] = temp["visit_id"] + "_updrs_" + str(u) + "_plus_" + str(m) + "_months"
/tmp/ipykernel_21/559174679.p

In [ ]:
def test_predictions(my_train, pro):
    proteins = pro.pivot(index=['visit_month', 'patient_id'], columns='UniProt', values='NPX')
    my_train = pd.merge(my_train, proteins, on=['visit_month', 'patient_id'], how='left')

    # Check which columns from features are not in my_train
    new_columns = [col for col in features if col not in my_train.columns]

    # Add the new columns to my_train with default value 0
    my_train[new_columns] = 0

    for column in list(my_train.columns):
      filled_data = replace_nan(my_train[column])
      my_train[column] = filled_data

    my_train[features[1:]] = scaler.transform(my_train[features[1:]])

    for u in target:
        # Here is where we will save the final results
        my_train['result_' + str(u)] = 0

        # Predict    
        X = my_train[features_selected[u]]
        my_train['result_' + str(u)] = np.floor(my_train[u])

    # Format for final submission
    result = pd.DataFrame()

    for m in [0, 6, 12, 24]:
        for u in [1, 2, 3, 4]:
            temp = my_train[["visit_id", "result_updrs_" + str(u)]]
            temp["prediction_id"] = temp["visit_id"] + "_updrs_" + str(u) + "_plus_" + str(m) + "_months"
            temp["rating"] = temp["result_updrs_" + str(u)]
            temp = temp[['prediction_id', 'rating']]

            result = result.append(temp)
    result = result.drop_duplicates(subset=['prediction_id', 'rating'])

    return result

test = test_predictions(train_clinical, train_proteins)

/tmp/ipykernel_21/1432767273.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp["prediction_id"] = temp["visit_id"] + "_updrs_" + str(u) + "_plus_" + str(m) + "_months"
/tmp/ipykernel_21/1432767273.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp["rating"] = temp["result_updrs_" + str(u)]
/tmp/ipykernel_21/1432767273.py:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append(temp)
/tmp/i

In [ ]:
smape(test['rating'], pred['rating'])

66.790727483331

In [ ]:
mean_squared_error(test['rating'], pred['rating'])

62.058413001912044

In [ ]:
r_squared(test['rating'], pred['rating'])

0.4805612414121566

In [ ]:
env = amp_pd_peptide_310.make_env()   # initialize the environment
iter_test = env.iter_test()  

for (test, test_peptides, test_proteins, sample_submission) in iter_test:
        
    result = get_predictions(test, test_proteins,test_peptides, model)

    env.predict(result)   # register your predictions

This version of the API is not optimized and should not be used to estimate the runtime of your code on the hidden test set.


/opt/conda/lib/python3.10/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
/tmp/ipykernel_21/559174679.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp["prediction_id"] = temp["visit_id"] + "_updrs_" + str(u) + "_plus_" + str(m) + "_months"
/tmp/ipykernel_21/559174679.p